In [ ]:
import numpy as np

#Initialization
ACTION_SPACE = ('U', 'D', 'L', 'R')
States=[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, 0),
 (1, 2),
 (1, 3),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3)]
REWARDS = {(0, 3): 1, (1, 3): -1}
actions = {
    (0, 0): ('D', 'R'),
    (0, 1): ('L', 'R'),
    (0, 2): ('L', 'D', 'R'),
    (1, 0): ('U', 'D'),
    (1, 2): ('U', 'D', 'R'),
    (2, 0): ('U', 'R'),
    (2, 1): ('L', 'R'),
    (2, 2): ('L', 'R', 'U'),
    (2, 3): ('L', 'U'),
    }

In [ ]:
def is_terminal(s):
    return s in [(0, 3),(1, 3)] 

In [ ]:
def print_values(V, rows,columns):
    for i in range(rows):
        print("---------------------------")
        for j in range(columns):
            v = V.get((i,j), 0)
            if v >= 0:
                print(" %.2f|" % v, end="")
            else:
                print("%.2f|" % v, end="") # -ve sign takes up an extra space
        print("")

In [ ]:
def get_next_state(s, a):
    # this answers: where would I end up if I perform action 'a' in state 's'?
    i, j = s[0], s[1]
    # if this action moves you somewhere else, then it will be in this dictionary
    if a in actions[(i, j)]:
        if a == 'U':
            i -= 1
        elif a == 'D':
            i += 1
        elif a == 'R':
            j += 1
        elif a == 'L':
            j -= 1
    return i, j

In [ ]:
### define transition probabilities
  # the key is (s, a, s'), the value is the probability
  # that is, transition_probs[(s, a, s')] = p(s' | s, a)
  # any key NOT present will considered to be impossible (i.e. probability 0)
transition_probs = {}
  # to reduce the dimensionality of the dictionary, we'll use deterministic
  # rewards, r(s, a, s')
  # note: you could make it simpler by using r(s') since the reward doesn't
  # actually depend on (s, a)
rewards = {}

for s in States:
    if not is_terminal(s):
        for a in ACTION_SPACE:
            s2 = get_next_state(s, a)
            transition_probs[(s, a, s2)] = 1
            if s2 in REWARDS:
                rewards[(s, a, s2)] = REWARDS[s2]
            else:
                rewards[(s, a, s2)] = 0

In [ ]:
### fixed policy ###
policy = {
    (2, 0): 'U',
    (1, 0): 'U',
    (0, 0): 'R',
    (0, 1): 'R',
    (0, 2): 'R',
    (1, 2): 'U',
    (2, 1): 'R',
    (2, 2): 'U',
    (2, 3): 'L',
  }

In [ ]:
def play_episode(policy,max_steps=20):
    start_states = list(actions.keys())
    start_idx = np.random.choice(len(start_states))
    s=start_states[start_idx]
    # keep track of all states and rewards encountered
    episodestates = [s]
    episoderewards = [0]

    steps = 0
    while not is_terminal(s):
        a = policy[s]
        next_s = get_next_state(s, a)
        r = rewards.get((s, a, next_s),0)
        # update states and rewards lists
        episodestates.append(next_s)
        episoderewards.append(r)
        s=next_s
       
        steps += 1
        if steps >= max_steps:
            break
    # update state
    # note: there is no need to store the final terminal state
    s = next_s
    # we want to return:
    # gamestates  = [s(0), s(1), ..., S(T)]
    # gamerewards = [R(0), R(1), ..., R(T)]

    return episodestates, episoderewards

In [ ]:
play_episode(policy,max_steps=20)

In [ ]:
# initialize V(s) and returns
GAMMA=0.9
V = {}
returns = {} # dictionary of state -> list of returns we've received
for s in States:
    if s in actions.keys():
        returns[s] = []
    else:
        # terminal state or state we can't otherwise get to
        V[s] = 0
# repeat
for t in range(100):
    # generate an episode using pi
    episodestates, episoderewards = play_episode(policy,max_steps=20)
    G = 0
    T = len(episodestates)
    for t in range(T - 2, -1, -1):
        s = episodestates[t]
        r = episoderewards[t+1]
        G = r + GAMMA * G # update return

      # we'll use first-visit Monte Carlo
        if s not in episodestates[:t]:
            returns[s].append(G)
            V[s] = np.mean(returns[s])

In [ ]:
print_values(V,3,4)

In [ ]:
for t in range(5 - 2, -1, -1):
    print(t)